Greetings! This is a python naoqi tutorial explaining the uses and protocol of naoqi's ALProxy module, which enables users to control Nao's with python scripts. Lets begin.

First and foremost, lets import ALProxy from naoqi so we can start working with it.

In [ ]:
from naoqi import ALProxy

To begin using all the modules installed on the Nao's, we first must create a "Robot" class, which requires an ip number and port number. One initializes this class to connect to a Nao that is currently connected to the wireless network your computer is connected to.

Here are some of the ip's of the NAOs in the Emergent Science Laboratory:
    #hedwig - 165.106.241.201
    #hoots - 165.106.241.202
    #nyctimene - 165.106.241.203
    #wol - 165.106.241.204

We initialize the modules/API's we would like to use by including them in the __init__() function. I included all the modules I used in my examples, but there exist many more useful ones in the Nao's latest documentation linked here: http://doc.aldebaran.com/2-1/naoqi/index.html

In [ ]:
class Robot:
    def __init__(self, ip, port):
        self.ip = ip
        self.port = port
        self.tts = ALProxy("ALTextToSpeech", ip, port)
        self.rbd = ALProxy("ALRedBallDetection", ip, port)
        #self.extr = ALProxy("ALExtractor", ip, port)
        #self.camera = ALProxy("ALPhotoCapture", ip, port) 
        self.visual = ALProxy("ALVideoDevice", ip, port)
        self.motion = ALProxy("ALMotion", ip, port)
        self.life = ALProxy("ALAutonomousLife", ip, port)
        self.posture = ALProxy("ALRobotPosture", ip, port)
        self.behave = ALProxy("ALBehaviorManager", ip, port)
        self.tracker = ALProxy("ALTracker", ip, port)
        self.tts.say("sure")
        self.life.setState("disabled")

Let's initialize a robot class now.

In [ ]:
wol = Robot("wol",9559)

ALTextToSpeech has a self explanatory function named say(), which has the Nao say the input string. Below is a function definition using it now.

In [ ]:
    def say(self, text):
        self.tts.say(text)

ALMotion contains many functions pretaining to the robot's individual movements of its joints and body parts. Before calling any movement functions however, one must turn on the motor for the specific joint(s) first with setStiffnesses(). This function takes the name of the joint or body part you want to address as a string, and the percentage of power you want the motor to run on as values between 0 and 1 as parameters. To prevent overheating, it is suggested to turn off the selected motors after you are done moving the Nao. 

Note: For these functions and many other modules' functions, adding a timed halt before the execution of the next line is necessary for functions to finish. For example, assuming a sitting position takes more than a few milliseconds for the Nao to complete, so a 10 second halt of the code is recommended for posture changes. This is why we must import python's time package to accomplish this. 

In [ ]:
import time

Here is a move_head() function with takes in a list of two floats, floats that range from -1 to 1, and set the robot's two head joints, HeadYaw and HeadPitch, to those values.

In [ ]:
    def move_head(self, newAngles):
        #newAngles = [HeadYaw value, HeadPitch value]
        self.motion.setStiffnesses("Head", 1.0)
        self.setAngles("HeadYaw", newAngles[0], 0.1)
        self.setAngles("HeadPitch", newAngles[1], 0.1)
        time.sleep(1.0)
        print ("done!")

I also wrote a function that sets the Nao's head at neutral position to more easily accomplish this.

In [ ]:
    def set_neutral(self):
        self.motion.setStiffnesses("Head", 1.0)
        self.motion.setAngles("HeadPitch", 0, 0.1)
        self.motion.setAngles("HeadYaw", 0, 0.1)
        time.sleep(3.0)
        self.motion.setStiffnesses("Head", 0)

The ALTracker module provides the user with a set of functions enabling the Nao to register and remember an object, and track the object with its head when the object is within its field of vision. registerTarget() takes the desired object to be remembered's name as a string and its diameter in meters as parameters. tracker() simply takes the name of your registered object as a string and begins to track it. I wrote a function track_ball() which compiles the two functions together.

In [ ]:
    def track_ball(self, target_name, diameter):
        self.tracker.registerTarget(target_name, diameter)
        time.sleep(3.0)
        temp = self.tracker.getRegisteredTargets()
        time.sleep(1.0)
        print temp
        temp2 = self.tracker.getMode()
        print "in mode: " + temp2
        self.motion.setStiffnesses("Head", 1.0)
        self.tracker.track(target_name)
        time.sleep(10.0)
        self.tracker.stopTracker()
        self.motion.setStiffnesses("Head", 0)

In [ ]:
import Image
import vision_definitions

subscribe() initializes the Nao's camera with specific settings passed through as parameters. It takes in the name of the image, the desired resolution you would like the Nao to take photos in, what colorspace to use, and at what frames per second (fps). These settings have been associated with ID numbers notated here: http://doc.aldebaran.com/2-1/naoqi/vision/alvideodevice-api.html#cameraresolution.

In [ ]:
#subscribe(pic_name, resolution, colorspace, fps)
testClient = wol.visual.subscribe("python_client", 8, 11, 5)

getImageRemote() takes the initialized camera variable as input and takes a photo with the desired settings. To construct this photo as a variable, we must use the function, .frombytes() from python's Image package. frombytes() requires an image's width, height, array, and desired colorspace to construct a photo, and we can grab this data by calling the first, second, and seventh item of the variable created with getImageRemote(), like so:

In [ ]:
testImage = wol.visual.getImageRemote(testClient)
testImageWidth = testImage[0]
testImageHeight = testImage[1]
testArray = testImage[6]
#frombytes(colorspace, (imageWidth, imageHeight), array)
testImg = Image.frombytes("RGB", (testImageWidth, testImageHeight), testArray)

From there, to actually save the photo as a file of a desired photo file type in the jupyter notebook's directory, we use Image's .save() function. It takes in the desired name of the photo plus the photo file type after it as a string, and the name of the photo file type as another string as parameters.

In [ ]:
testImg.save("testPhoto123.jpeg", "JPEG")

from unsubscribe() simply uninitializes a camera variable by passing the variable into this function as a parameter.

In [ ]:
wol.visual.unsubscribe(testClient)

Here is a function save_photo() I wrote that combines all of these functions together to take a photo with the Nao and save it with the desired photo name as a string as input.

In [ ]:
    def save_photo(self, img_name): #takes 640x480 photo and saves to .py file directory
        print "taking photo: " + img_name
        vidClient = self.visual.subscribe("python_client", 8, 11, 5) #initialize cam with settings
        naoImage = self.visual.getImageRemote(vidClient) #taking photo
        time.sleep(0.5)
        self.visual.unsubscribe(vidClient) #un-initialize
        imageWidth = naoImage[0]
        imageHeight = naoImage[1]
        array = naoImage[6]
        img = Image.frombytes("RGB", (imageWidth, imageHeight), array)
        img.save(img_name + ".jpeg", "JPEG")
        print "photo " + img_name + " saved!"

For something a bit more complicated, here is the function, recognize_n_act(), which utilizes functions from ALMotion, ALVideoDevice, and ALTracker and showcases how all these modules can work in unison to accomplish a task. It set's the Nao's head to neutral, saves an image, then tracks the ball, says its found it, return the joint angles of its head, and stops tracking the ball. 

In [ ]:
    def recognize_n_act(self, target_name, img_name):
        #sets nao's head to neutral, saves an image, then tracks the ball,
        #says its found it, return the joint angles of its head, and stops tracking
        self.set_neutral()
        time.sleep(1.0)
        self.save_photo(img_name)
        self.motion.setStiffnesses("Head", 1.0)
        self.tracker.track(target_name)
        time.sleep(1.5)
        self.tts.say("found " + target_name)
        temp3 = str(self.motion.getAngles("HeadYaw", True))[1:-1]
        temp4 = str(self.motion.getAngles("HeadPitch", True))[1:-1]
        fp = open("naoheadangles.txt", "a")
        fp.write(temp3 + ", " + temp4 + "\n") #writing angles into .txt file
        fp.close()
        time.sleep(0.5)
        self.tracker.stopTracker()
        self.motion.setStiffnesses("Head", 0)
        self.set_neutral()
        #does all the work for me :3

In addition, I wrote an additional function, save_multiple_photos(), which calls recognize_n_act() a set number of times that you enter in as input in the function, and pauses and has the Nao ask the user to move the ball to a new location before it takes the next photo. This function comes in handy in the "Intro to Conx" tutorial for collecting training_set data.

In [ ]:
    def save_multiple_photos(self, target_name, img_name, amount):
        counter = "1"
        for index in range(0, amount):
            self.recognize_n_act(target_name, img_name + counter)
            if index == amount:
                break
            self.tts.say("move " + target_name + "to a new location")
            time.sleep(2.0)
            self.tts.say("ready? okay")
            counter = str(int(counter) + 1)

This tutorial only goes through a few modules and now they work, however there exist many other modules available for use. Here is a small key they compiles the models for the most used functions in the tutorial:

#functions you'd like to know:
    #getAngles(names, bool useSensors)
    #setAngles(names, angles, fractionMaxSpeed)
    #changeAngles(names, changes, fractionMaxSpeed)
    #getSubscribersInfo() -- Gets the list of parameters for all the current subscribers (name, period and precision)
    #getPostureList()
    #goToPosture(name, speed)
    #setState(name)
    #frombytes(colorspace, (imageWidth, imageHeight), array)
    #subscribe(pic_name, resolution, colorspace, fps)
        #2 is 640x480, 3 is 1280x960, 8 is 40x30
        #supports up to 30 fps
        #dont change colorspace, its on standard RGB colorspace